# 기존 서비스 --> 대화형 추천으로 변경
1. 추천풀 및 메뉴 벡터 DB 생성 <-- 해당 노트북
2. 사용자 발화 기반 개인화 추천 API 개발
3. 데모 수정: 대화형 UI 및 Function Calling, 멀티턴 대화 기능 추가
- OpenAI Function Calling 복습

## 추천풀 및 메뉴 벡터 DB 생성
### Restaurant Info 정보 가져오기

- 현재 포맷

In [ ]:
'''
"1367138" : {
    "restaurant": "88곱창-분당점",
    "url": "https://www.yogiyo.co.kr/mobile/#/1367138/",
    "reviews": [
        {"menus": "menu1,menu2,menu3", "review_text": "맛있음"},
        {"menus": "menu1,menu3,menu5", "review_text": "굿"}
    ]
}
'''

In [1]:
import os
import urllib

import certifi
from pymongo.server_api import ServerApi
from pymongo import MongoClient

from dotenv import load_dotenv

load_dotenv("../res/.env")

uri = os.environ['MONGODB_URI']
client = MongoClient(uri, server_api=ServerApi('1'), tlsCAFile=certifi.where())
db = client.restaurant_db
collection = db.restaurant_info

# restaurants_info = list(collection.find({}, {'_id': False}))
restaurants_info = list(collection.find({}))
restaurants_info

[{'_id': '1031899',
  'restaurant': '버거킹-판교유스페이스점',
  'reviews': [{'menus': '통새우와퍼세트/1(사이드변경(프렌치프라이R - 기본),음료변경(코카-콜라R（변경 없음 ）)),와퍼/1',
    'review_text': '든든하게 먹었습니다아'},
   {'menus': '통새우와퍼세트/1(사이드변경(프렌치프라이L),음료변경(스프라이트L)),롱치킨버거/1',
    'review_text': '매번 자주 시켜먹는 곳인데 말이 필요 없습니다..너무 맛있습니다.'},
   {'menus': '몬스터와퍼세트/1(사이드변경(프렌치프라이R - 기본),음료변경(스프라이트R)),몬스터와퍼세트/1(사이드변경(프렌치프라이R - 기본),음료변경(코카-콜라R（변경 없음 ）)),통새우와퍼세트/1(사이드변경(프렌치프라이R - 기본),음료변경(코카-콜라R（변경 없음 ）))',
    'review_text': '회사에서 사진촬영이 금지여서 대충 찍었지만 맛있게 잘 먹었습니다'},
   {'menus': '치킨킹 세트/1(사이드변경(프렌치프라이R - 기본),음료변경(스프라이트 제로R)),와퍼/1',
    'review_text': '사진엔 없지만 기본와퍼도 맛있네요'},
   {'menus': '콰트로치즈와퍼 세트/1(사이드변경(프렌치프라이R - 기본),음료변경(코카-콜라R（변경 없음 ）)),통새우와퍼세트/1(사이드변경(프렌치프라이R - 기본),음료변경(코카-콜라R（변경 없음 ）)),통새우와퍼세트/1(사이드변경(프렌치프라이R - 기본),음료변경(아이스아메리카노))',
    'review_text': '주문한 버거 3개 중 2개만 와서 누락분 다시 보내달라고 연락한 지 30분이 넘었는데, 최대한 빨리 보내준다더니 아직도 안 오고 있는 상황 허허허...와이프와 아이는 이미 다 먹었고, 저는 감튀만 먹고 있습니다. 이제 벨이 울리네요..'},
   {'menus': '불끈버거 맥시멈 세트/1(사이드변경(프렌치프라이R - 기본),음료

## menu_db 만들기
- menu split
- filter review_related

In [2]:
'''
"1367138_1" : {
    "restaurant": "88곱창-분당점",
    "menu": "수박주스x3",
    "url": "https://www.yogiyo.co.kr/mobile/#/1367138/",
    "keywords": ["해장", "숙취해소"],
    "embeddings": [
        ...
    ]
}
'''

'\n"1367138_1" : {\n    "restaurant": "88곱창-분당점",\n    "menu": "수박주스x3",\n    "url": "https://www.yogiyo.co.kr/mobile/#/1367138/",\n    "keywords": ["해장", "숙취해소"],\n    "embeddings": [\n        ...\n    ]\n}\n'

In [3]:
KEYWORDS_BLACKLIST = ['리뷰', 'zㅣ쀼', 'ZI쀼', 'Zl쀼', '리쀼', '찜', '이벤트', '추가', '소스']
KEYWORDS_CONTEXT = [
    '해장', '숙취',
    '다이어트'
]

In [4]:
def is_valid_menu(menu_name):
    return True if not any(keyword in menu_name for keyword in KEYWORDS_BLACKLIST) else False

is_valid_menu('[리쀼] 수박 추가')

False

In [5]:
def extract_keywords(review_text):
    keywords = []

    for word in review_text.split():
        if any(keyword in word for keyword in KEYWORDS_CONTEXT):
            keywords.append(word)
    return keywords

extract_keywords('완전 맛있었어요. 역시 해장에는 수박주스가 답이네요.')

['해장에는']

In [6]:
KEYWORDS_BLACKLIST = ['리뷰', 'zㅣ쀼', 'ZI쀼', 'Zl쀼', '찜', '이벤트', '추가']
KEYWORDS_CONTEXT = [
    '해장', '숙취',
    '다이어트'
]

menu_name2id = {}
menu_db = {}

unique_menus = set()
for index, restaurant in enumerate(restaurants_info):
    menu_name2id = {}
    for reviews in restaurant['reviews']:
        menus = reviews["menus"].split(',')
        review_text = reviews["review_text"]

        # 리뷰에 컨텍스트 관련 키워드 있는 지 확인
        keywords = extract_keywords(review_text)

        # 리뷰에 컨텍스트 관련 키워드 없으면 해당 리뷰 스킵
        if keywords == []:
            continue

        for menu in menus:
            menu_name = menu.split('/')[0]
            # 리뷰 이벤트 관련 메뉴 필터링
            if is_valid_menu(menu_name):
                if not menu_name in menu_name2id:
                    menu_idx = len(menu_name2id)
                    menu_id = '_'.join([restaurant['_id'], str(menu_idx)])
                    menu_name2id[menu_name] = menu_id

                    # Menu DB instances format
                    menu_db[menu_id] = {
                        "restaurant": restaurant["restaurant"],
                        "menu": menu_name,
                        "url": restaurant["url"],
                        "keywords": menu_name.split(),
                        "embeddings": None
                    }
                else:
                    menu_id = menu_name2id[menu_name]

                menu_db[menu_id]["keywords"].extend(keywords)

In [7]:
len(menu_db)


5

In [9]:
import sys
sys.path.insert(0, "..")

from utils import get_embedding

for menu_id in menu_db:
    keywords = menu_db[menu_id]["keywords"] + [menu_db[menu_id]["menu"]]
    embedding = get_embedding(" ".join(keywords), model='text-embedding-3-large')
    menu_db[menu_id]["embeddings"] = embedding

In [10]:
db = client.menu_db
collection = db.menu_info

In [11]:
for menu_id in menu_db:
    result = collection.update_one({"_id": menu_id}, {"$set": menu_db[menu_id]}, upsert=True)